In [8]:
import pandas as pd
import tskit as tsk
import numpy as np
from multiprocessing import Pool
from datetime import datetime
import json
import os

# %run -i "../isRecipMonophyletic.py" TO DEFINE NEW FUNCTION

perFile = True

if (perFile) :
	inputdir = '../../data/real/'
	inputs = [ # List of input .trees files
		"cgal379.trees",
		"Cgale_87_dated.trees",
		"Cgale_952_dated.trees"
	]
else :
	inputdir = 'B:\\storage\\general-elias\\Code\\TS_FILES'
	inputs = os.listdir(inputdir) # List of input .trees files

startTime = datetime.now() # For performance measurement

In [9]:
def loadFile(filename) : # Does all the necessary imports on a single file
    sqObj = tsk.load(inputdir + filename)

    populations = str(sqObj.tables.populations.asdict()) # For later check, see cell 4

    pop_by_node = pd.DataFrame({ # Only one of these will be kept, thus the check above
        "pop": [sqObj.tables.nodes[leaf].population for leaf in sqObj.samples()], # Get the population of each leaf
    })

    # Create standalone pandas DataFrame for easier manipulation of data than tskit's tables...

    pd_sequence = pd.DataFrame(
        {
            'file': filename, # Gets the originating filename for future reference
            'span': [tree.span for tree in sqObj.trees()],
            'bounds': [(tree.interval.left, tree.interval.right) for tree in sqObj.trees()], # Loads the tree's bound in a tuple
            'treeObj' : sqObj.aslist()
        },
        index=[tree.index for tree in sqObj.trees()]
    )

    return sqObj, populations, pop_by_node, pd_sequence # All the information needed from a tree sequence

In [10]:
files = pd.DataFrame({ # Initiates the DataFrame with the input files
    'file': inputs,
})

# Fill the files DataFrame with the necessary information
files[['sqObj', 'populations', 'pop_by_node', 'pd_sequence']] = files.apply(lambda row: loadFile(row['file']), axis=1, result_type="expand")

In [11]:
# *The* check I was talking about earlier. If the populations are not the same, that's a problem...
if files.populations.nunique() != 1:
    raise ValueError("All the files do not have the same populations. Are you sure this is the same species ?")
else :
    pop_by_node = files.pop_by_node[0]

In [12]:
pop_groups = [ # Group the populations according to real data
    [0, 1], 
    [2, 3]
]

In [13]:
# TOOL FOR AUTOMATIC COLORING (I know, it shouldn't really be here...)

# Define a list of predefined colors
predefined_colors = ["red", "blue", "green", "yellow", "orange", "purple", "brown", "pink"] # Maximum 8 populations...

# Initialize node_colours dictionary
node_colours = {}

# Example :
exSq = files.iloc[0].sqObj

for node_index, node in enumerate(exSq.tables.nodes):
	if (node.flags & tsk.NODE_IS_SAMPLE) != 0: # If node is a sample
		
		# Assign color from predefined list, cycling through colors if necessary
		color = predefined_colors[node.population % len(predefined_colors)]
		node_colours[node_index] = color



In [14]:
# Generates a global pd_sequence from all the files to treat them as one.
globalTS = pd.concat(files.pd_sequence.to_list(), ignore_index=True)

### Bootstrap approach

In [29]:
n_threads = 0 # Threads to use for multiprocessing, 0 means no multiprocessing
n_samples = 0 # 0 means no bootstrapping

def f(i) : # Define util to be run in parallel
    np.random.seed(i + np.random.randint(0, 10000)) # Necessary, because otherwise all workers will have the same seed
    bootstrap = globalTS.sample(n=int(len(globalTS) * 0.9), replace=True) # Use pandas sample method to take random trees in the sequence for monophyly test
    bootstrap['monophyletic'] = bootstrap.apply(lambda x: isRecipMonophyletic(x.treeObj, pop_by_node, pop_groups), axis=1) # Runs the test
    return bootstrap[bootstrap['monophyletic'] == True]['span'].sum() / bootstrap['span'].sum() # Gets the percentage of the sample trees that is monophyletic

if n_threads > 0 :
    pool = Pool(n_threads) # Create a pool of 2 workers
    percentages = np.array(pool.map(f, range(n_samples))) * 100 # Run bootstrap twice in parallel
else :
    percentages = np.array(list(map(f, range(n_samples)))) * 100

### Monophyly test for the output

In [30]:
# Runs the reciprocally monophyletic test on each tree and saves the boolean result in the monophyletic column

globalTS['monophyletic'] = globalTS.apply(lambda x: isRecipMonophyletic(x.treeObj, pop_by_node, pop_groups), axis=1)
singlepercentage = globalTS[globalTS['monophyletic'] == True]['span'].sum() / globalTS['span'].sum() * 100

### Export results

In [31]:
# Export sampling data

if n_samples > 0 :
    sampling = {
		"meanSamplesPercentage": percentages.mean(),
		"samplesStdDev": percentages.std(),
		"confidence_interval": {
			"bounds": [5, 95],
			"lower": np.percentile(percentages, 5),
			"upper": np.percentile(percentages, 95)
		},
	}
else :
    sampling = {}

In [32]:
endTime = datetime.now()

In [33]:
# Define output file format

output = {
    "input": {
        "dir": inputdir,
        "files": inputs
	},
    "test": "reciprocal_monophyly",
    "description": "Reciprocal monophyly test on the tree sequence",
    "analysis_settings": {
        "bootstrap_samples": n_samples,
        "pop_groups": pop_groups,
    },
    "perf" : {
        "multiprocessing": n_threads > 0,
        "threads": n_threads,
        "start": startTime,
        "end": endTime,
        "duration": endTime - startTime
    },
    "result": {
        "sampling": sampling,
        "raw": {
			"positive": int((globalTS['monophyletic'] == True).sum()),
			"negative": int((globalTS['monophyletic'] == False).sum()),
			"total_trees": len(globalTS),
			"percentage": singlepercentage,
			"details": globalTS[['file', 'bounds', 'span', 'monophyletic']].to_dict(orient='records')
		},
	}
}

In [34]:
json.dump(output, default=str, indent=4, fp=open("../../data/out/reciprocal_monophyly_result.json", "w"))